In [1]:
!pip install pyspark==3.0.0 koalas==1.0.1 pyarrow==0.15.1 scikit-learn==0.23.1

In [2]:
!rm log1* && wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log1.txt

--2020-07-02 10:05:45--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19954 (19K) [text/plain]
Saving to: ‘log1.txt’

100%[======================================>] 19,954      --.-K/s   in 0.001s  

2020-07-02 10:05:45 (13.0 MB/s) - ‘log1.txt’ saved [19954/19954]



In [3]:
!echo "ts,rssi"> log1.csv
!cat log1.txt |awk  '{print $4","$11}' |tee -a  log1.csv

07:49:45,-86
07:49:50,-84
07:49:55,-86
07:50:00,-87
07:50:05,-77
07:50:10,-72
07:50:15,-66
07:50:20,-73
07:50:25,-67
07:50:30,-74
07:50:35,-68
07:50:40,-52
07:50:45,-62
07:50:50,-76
07:50:55,-80
07:51:00,-69
07:51:05,-69
07:51:10,-69
07:51:15,-69
07:51:20,-69
07:51:25,-67
07:51:30,-69
07:51:35,-68
07:51:40,-69
07:51:45,-69
07:51:50,-69
07:51:55,-76
07:52:00,-69
07:52:05,-77
07:52:10,-53
07:52:15,-77
07:52:20,-69
07:52:25,-69
07:52:30,-69
07:52:35,-69
07:52:40,-77
07:52:45,-77
07:52:50,-62
07:52:55,-69
07:53:00,-54
07:53:05,-83
07:53:10,-91
07:53:15,-90
07:53:20,-90
07:53:25,-100
07:53:30,-89
07:53:35,-87
07:53:40,-88
07:53:45,-89
07:53:50,-87
07:53:55,-88
07:54:00,-88
07:54:05,-90
07:54:10,-92
07:54:15,-88
07:54:20,-87
07:54:25,-79
07:54:30,-77
07:54:35,-88
07:54:40,-89
07:54:45,-88
07:54:50,-78
07:54:55,-88
07:55:00,-89
07:55:05,-91
07:55:10,-100
07:55:15,
07:55:20,
07:55:25,-99
07:55:30,
07:55:35,
07:55:40,
07:55:45,-100
07:55:50,
07:55:55,-98
07:56:00,
07:56:05,
07:56:10,-97
07:56:1

In [4]:
!rm markers* && wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers1.csv

--2020-07-02 10:05:48--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers1.csv’

100%[======================================>] 151         --.-K/s   in 0s      

2020-07-02 10:05:48 (5.82 MB/s) - ‘markers1.csv’ saved [151/151]



In [5]:
!head markers1.csv

ts_start,ts_stop,location,comment
07:30:05,07:53:05,0,sofa
07:53:10,07:55:10,1,tuer
07:55:15,07:57:15,2,schlafzimmertuer
07:57:20,07:59:06,3,olivenoel


In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [7]:
data = spark.read.option('header','true').csv('log1.csv')
markers = spark.read.option('header','true').csv('markers1.csv')

In [8]:
#convert ts_string to ts_timestamp
from pyspark.sql.functions import to_timestamp
data = data.withColumn("ts_p", to_timestamp("ts", "HH:mm:ss"))
markers = markers.withColumn("ts_start_p", to_timestamp("ts_start", "HH:mm:ss"))
markers = markers.withColumn("ts_stop_p", to_timestamp("ts_stop", "HH:mm:ss"))
data.show()

+--------+----+-------------------+
|      ts|rssi|               ts_p|
+--------+----+-------------------+
|07:49:45| -86|1970-01-01 07:49:45|
|07:49:50| -84|1970-01-01 07:49:50|
|07:49:55| -86|1970-01-01 07:49:55|
|07:50:00| -87|1970-01-01 07:50:00|
|07:50:05| -77|1970-01-01 07:50:05|
|07:50:10| -72|1970-01-01 07:50:10|
|07:50:15| -66|1970-01-01 07:50:15|
|07:50:20| -73|1970-01-01 07:50:20|
|07:50:25| -67|1970-01-01 07:50:25|
|07:50:30| -74|1970-01-01 07:50:30|
|07:50:35| -68|1970-01-01 07:50:35|
|07:50:40| -52|1970-01-01 07:50:40|
|07:50:45| -62|1970-01-01 07:50:45|
|07:50:50| -76|1970-01-01 07:50:50|
|07:50:55| -80|1970-01-01 07:50:55|
|07:51:00| -69|1970-01-01 07:51:00|
|07:51:05| -69|1970-01-01 07:51:05|
|07:51:10| -69|1970-01-01 07:51:10|
|07:51:15| -69|1970-01-01 07:51:15|
|07:51:20| -69|1970-01-01 07:51:20|
+--------+----+-------------------+
only showing top 20 rows



In [9]:
data.createOrReplaceTempView('data')
markers.createOrReplaceTempView('markers')

In [10]:
data = spark.sql('select * from data d inner join markers m on d.ts_p>=m.ts_start_p and d.ts_p<=m.ts_stop_p')

In [12]:
import databricks.koalas as ks
kdf = data.to_koalas()

In [13]:
kdf = kdf.dropna()

In [14]:
pdf = kdf[['rssi','location']].toPandas()

In [15]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(pdf[['rssi']], pdf[['location']])

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '1', '1', '1', '1', '3', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1',
       '3', '3', '3', '3', '3', '1', '3', '3', '3', '3', '1', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '1'],
      dtype=object)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(pdf[['location']], clf.predict(pdf[['rssi']]))

0.7475728155339806